# Advent of Code: Day 14

In [96]:
using Base.Iterators
using Combinatorics
using DataStructures
using Match
using StatsKit
using Underscores

#fn = "ExampleInput.txt"
fn = "SolutionInput.txt"

lines = readlines(fn)
template = lines[1]
insertions = @_ lines[3:length(lines)] |> map(split(_, " -> "), __) |> Dict

Dict{SubString{String}, SubString{String}} with 100 entries:
  "FN" => "V"
  "PO" => "C"
  "CC" => "V"
  "KF" => "O"
  "BC" => "P"
  "BV" => "K"
  "OP" => "K"
  "PF" => "C"
  "NB" => "S"
  "BB" => "P"
  "NH" => "H"
  "PP" => "B"
  "VN" => "S"
  "SF" => "H"
  "CS" => "C"
  "PN" => "N"
  "CF" => "S"
  "NO" => "N"
  "KV" => "B"
  "OS" => "N"
  "VH" => "P"
  "PS" => "H"
  "BS" => "N"
  "NP" => "V"
  "HO" => "N"
  ⋮    => ⋮

## Part 1

In [97]:
function executeinsertions(template, insertions)
  template_next = collect(template)
  for i ∈ 1:length(template) - 1
    pair = template[i:i + 1]
    if haskey(insertions, pair)
      new_char = insertions[pair][1]
      length_offset = length(template_next) - length(template)
      insert!(template_next, length_offset + i + 1, new_char)
    end
  end
  join(template_next)
end

counts = @_ template                |>
            accumulate((template, step) -> begin
              (step, executeinsertions(template[2], insertions))
            end, 1:10; init=(0,__)) |>
            last                    |>
            countmap(collect(__[2]))

countvalues = values(counts) |> collect

maximum(countvalues) - minimum(countvalues)

2068

## Part 2

In [98]:
template_pairs = Dict()

for i ∈ 1:length(template) - 1
  pair = join(template[i:i + 1])
  if haskey(template_pairs, pair)
    template_pairs[pair] += 1
  else
    template_pairs[pair] = 1
  end
end

function executeinsertions2(template, insertions)
  template_next = Dict(template)
  for (pair, count) ∈ pairs(template)
    if !haskey(insertions, pair); continue end

    new_char = insertions[pair]
    new_pair1 = "$(pair[1])$(new_char[1])"
    new_pair2 = "$(new_char[1])$(pair[2])"

    template_next[pair] -= count
    for new_pair ∈ [new_pair1, new_pair2]
      if haskey(template_next, new_pair)
        template_next[new_pair] += count
      else
        template_next[new_pair] = count
      end
    end
  end
  filter!(entry -> entry[2] > 0, template_next)
  template_next
end

function countelementsmap(template)
  elementcounts = DefaultDict(0)
  for (pairs, count) ∈ pairs(template)
    for element ∈ collect(pairs)
      elementcounts[element] += count
    end
  end
  for (element, count) ∈ pairs(elementcounts)
    elementcounts[element] = ceil(Int, count / 2)
  end
  elementcounts
end

counts2 = @_ template_pairs                |>
             accumulate((template, step) -> begin
               (step, executeinsertions2(template[2], insertions))
             end, 1:40; init=(0,__)) |>
             last                    |>
             countelementsmap(__[2])

countvalues2 = values(counts2) |> collect

maximum(countvalues2) - minimum(countvalues2)

2158894777814